In [ ]:
#!pip install chromadb

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [ ]:
loader = TextLoader("../data/US_constitution.txt")
documents = loader.load()

documents[0].page_content

In [ ]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=256)
docs = text_splitter.split_documents(documents)

In [ ]:
embedding_function = OpenAIEmbeddings()
db = Chroma.from_documents(documents, embedding_function, persist_directory="./chroma-db")
db.persist()


In [65]:
llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo')
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=db.as_retriever(search_kwargs={"k": 1}))


In [67]:
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.ContextualCompressionRetriever").setLevel(logging.DEBUG)

question = "What is the first amendment?"
compressed_docs = compression_retriever.get_relevant_documents(question)
                                                                # kwargs={'reduce_k_below_max_tokens':True})

d:\SameerWork\VSCode_Workspace\langchain-experiments\.venv\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens. However, your messages resulted in 9070 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}